In [ ]:
#Library for data manipulation and analysis
import pandas as pd

In [ ]:
pwd

In [ ]:
import os

In [ ]:
import numpy as np
from numpy.random import seed
from numpy.random import randn



from sklearn import linear_model
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.graphics.gofplots import qqplot
import statsmodels.api as sm

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')

import seaborn as sns
sns.set(style="whitegrid")

import math
%config IPCompleter.greedy=True

import csv
import time

import scipy.stats as stats

In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [ ]:
##1.	Read the ‘csv’ file provided 

In [ ]:
dataset = pd.read_csv("../input/predicting-the-sales-of-products-of-a-retail-chain/train_data.csv", names=['date','product_identifier','department_identifier','category_of_product','outlet','state','sales'],sep=',', header=0)
prod_price = pd.read_csv("../input/predicting-the-sales-of-products-of-a-retail-chain/product_prices.csv",sep=',', header=0,names=['outlet','product_identifier','week_id','sell_price'])
date_week=pd.read_csv("../input/predicting-the-sales-of-products-of-a-retail-chain/date_to_week_id_map.csv",sep=',',header=0, names=['date','week_id'])

In [ ]:
##a.	Get the dimensions of the data

In [ ]:
dataset.shape, prod_price.shape, date_week.shape

In [ ]:
##merge csv files

In [ ]:
df=pd.merge(prod_price,date_week, on=['week_id'], how='inner')
dataset=pd.merge(dataset,df, on=['date','product_identifier','outlet'], how='inner')
dataset

In [ ]:
##b.	Get the summary

In [ ]:
dataset.describe()

In [ ]:
##c.	Print the first 5 rows and last 5 rows.

In [ ]:
dataset.head(5)

In [ ]:
##c.	Print the first 5 rows and last 5 rows.
dataset.tail(5)

In [ ]:
##5.	Verify if there are any null values in each of the columns. (don’t impute now).

In [ ]:
dataset.isnull().sum()

In [ ]:
##d.	Explore the data types of each column.
dataset.dtypes

In [ ]:
##converting as category
col = ['category_of_product', 'state']
dataset[col] = dataset[col].astype('category')

In [ ]:
##e.	Understand if all the columns are in appropriate datatypes or not.
dataset.dtypes

In [ ]:
##check for insights using scatterplot
def scatter(dataset, column):
    plt.figure()
    plt.scatter(dataset[column] , dataset['sales'])
    plt.ylabel('sales')
    plt.xlabel(column)

In [ ]:
scatter(dataset, 'product_identifier')
scatter(dataset, 'department_identifier')
scatter(dataset, 'category_of_product')
scatter(dataset, 'outlet')
scatter(dataset, 'state')
scatter(dataset, 'week_id')
scatter(dataset, 'sell_price')

In [ ]:
fig = plt.figure(figsize=(18, 14))
corr = dataset.corr()
c = plt.pcolor(corr)
plt.yticks(np.arange(0.5, len(corr.index), 1), corr.index)
plt.xticks(np.arange(0.5, len(corr.columns), 1), corr.columns)
fig.colorbar(c)

In [ ]:
#vars() function in Python,vars() method takes only one parameter and that too is optional,
#It takes an object as a parameter which may be can a module, a class, an instance, or any object having __dict__ attribute.
sns.pairplot(dataset, vars=['product_identifier','department_identifier','category_of_product','outlet','state','sales'])

In [ ]:
for name, group in dataset.groupby(["outlet", "department_identifier"]):
    plt.title(name)
    plt.scatter(range(len(group)), group["sales"])
    plt.show()
    break

In [ ]:
##use label encoder-LabelEncoder encode labels with a value between 0 and n_classes-1 where n is the number of distinct labels
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
dataset['state_encoded'] = LabelEncoder().fit_transform(dataset['state'])
dataset[['state', 'state_encoded']]

In [ ]:
#### Maharastra=1; Kerala = 0; Telangana=2

In [ ]:
dataset['cat_prod_encoded'] = LabelEncoder().fit_transform(dataset['category_of_product'])
dataset[['category_of_product', 'cat_prod_encoded']]

In [ ]:
#### drinks_and_food=0; fast_moving_consumer_goods=1; others=2

In [ ]:
dataset['Month'] = pd.to_datetime(dataset['date']).dt.month
dataset = dataset.drop(columns=["date","week_id","state","category_of_product"])
dataset

In [ ]:
##using different machine learning algorithm

In [ ]:
def knn():
    knn = KNeighborsRegressor(n_neighbors=10)
    return knn

def extraTreesRegressor():
    clf = ExtraTreesRegressor(n_estimators=100,max_features='auto', verbose=1, n_jobs=1)
    return clf

def randomForestRegressor():
    clf = RandomForestRegressor(n_estimators=100,max_features='log2', verbose=1)
    return clf

def svm():
    clf = SVR(kernel='rbf', gamma='auto')
    return clf

def predict_(m, test_x):
    return pd.Series(m.predict(test_x))

def model_():
#     return knn()
    return extraTreesRegressor()
#     return svm()
#     return nn()
#     return randomForestRegressor()    

def train_(train_x, train_y):
    m = model_()
    m.fit(train_x, train_y)
    return m

def train_and_predict(train_x, train_y, test_x):
    m = train_(train_x, train_y)
    return predict_(m, test_x), m


In [ ]:
def calculate_error(test_y, predicted, weights):
    return mean_absolute_error(test_y, predicted, sample_weight=weights)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
##K-Folds cross-validator
#Provides train/test indices to split data in train/test sets. Split dataset into k consecutive folds (without shuffling by default).
kf = KFold(n_splits=5)
splited = []
# dataset2 = dataset.copy()
for name, group in dataset.groupby(["outlet", "department_identifier"]):
    group = group.reset_index(drop=True)
    trains_x = []
    trains_y = []
    tests_x = []
    tests_y = []
    if group.shape[0] <= 5:
        f = np.array(range(5))
        np.random.shuffle(f)
        group['fold'] = f[:group.shape[0]]
        continue
    fold = 0
    for train_index, test_index in kf.split(group):
        group.loc[test_index, 'fold'] = fold
        fold += 1
    splited.append(group)

splited = pd.concat(splited).reset_index(drop=True)

In [ ]:
splited

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
best_model = None
error_cv = 0
best_error = np.iinfo(np.int32).max
for fold in range(5):
    dataset_train = splited.loc[splited['fold'] != fold]
    dataset_test = splited.loc[splited['fold'] == fold]
    train_y = dataset_train['sales']
    train_x = dataset_train.drop(columns=['sales', 'fold'])
    test_y = dataset_test['sales']
    test_x = dataset_test.drop(columns=['sales', 'fold'])
    print(dataset_train.shape, dataset_test.shape)
    predicted, model = train_and_predict(train_x, train_y, test_x)
    weights = test_x['outlet'].replace(True, 5).replace(False, 1)
    error = calculate_error(test_y, predicted, weights)
    error_cv += error
    print(fold, error)
    if error < best_error:
        print('Find best model')
        best_error = error
        best_model = model
error_cv /= 5

In [ ]:
if error < best_error:
        print('Find best model')
        best_error = error
        best_model = model

In [ ]:
print(best_error)

In [ ]:
##Decision tree
print(best_model)

In [ ]:
##Using test data

In [ ]:
dataset_test = pd.read_csv("../input/predicting-the-sales-of-products-of-a-retail-chain/test_data.csv", names=['id','date','product_identifier','department_identifier','category_of_product','outlet','state'],sep=',', header=0)
prod_price = pd.read_csv("../input/predicting-the-sales-of-products-of-a-retail-chain/product_prices.csv",sep=',', header=0,names=['outlet','product_identifier','week_id','sell_price'])
date_week=pd.read_csv("../input/predicting-the-sales-of-products-of-a-retail-chain/date_to_week_id_map.csv",sep=',',header=0, names=['date','week_id'])

In [ ]:
df=pd.merge(prod_price,date_week, on=['week_id'], how='inner')
dataset_test=pd.merge(dataset_test,df, on=['date','product_identifier','outlet'], how='inner')
dataset_test

In [ ]:
##checking datatypes for test data
dataset_test.dtypes

In [ ]:
col = ['category_of_product', 'state']
dataset_test[col] = dataset_test[col].astype('category')

In [ ]:
dataset_test.dtypes

In [ ]:
##label encoder to convert categorical data, or text data, into numbers, which our predictive models can better understand.
dataset_test['state_encoded'] = LabelEncoder().fit_transform(dataset_test['state'])
dataset_test[['state', 'state_encoded']]

In [ ]:
#### Maharastra=1; Kerala = 0; Telangana=2

In [ ]:
dataset_test['cat_prod_encoded'] = LabelEncoder().fit_transform(dataset_test['category_of_product'])
dataset_test[['category_of_product', 'cat_prod_encoded']]

In [ ]:
#### drinks_and_food=0; fast_moving_consumer_goods=1; others=2

In [ ]:
dataset_test['Month'] = pd.to_datetime(dataset_test['date']).dt.month
dataset_test = dataset_test.drop(columns=["date","week_id","state","category_of_product","id"])
dataset_test

In [ ]:
predicted_test = best_model.predict(dataset_test)

In [ ]:
dataset_test['sales'] = predicted_test

dataset_test['id'] = dataset_test['outlet'].astype(str) + '_' +  dataset_test['department_identifier'].astype(str)
dataset_test = dataset_test[['id', 'sales']]
dataset_test = dataset_test.rename(columns={'id': 'Id', 'sales': 'sales'})

In [ ]:
##index=false--avoid pandas creating an index in saved csv file
dataset_test.to_csv('final_output.csv', index=False)